<a href="https://colab.research.google.com/github/noamsiegel/Projects/blob/main/Neural_Network_SMS_Text_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install --upgrade tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load the train and test data
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_data['message'])

# Convert text to sequence
train_sequences = tokenizer.texts_to_sequences(train_data['message'])
train_padded = pad_sequences(train_sequences, maxlen=100, truncating='post', padding='post')

test_sequences = tokenizer.texts_to_sequences(test_data['message'])
test_padded = pad_sequences(test_sequences, maxlen=100, truncating='post', padding='post')

# Define the model architecture
model = keras.Sequential([
    keras.layers.Embedding(input_dim=10000, output_dim=16, input_length=100),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
history = model.fit(train_padded, train_data['label']=='spam', epochs=10, 
                    validation_data=(test_padded, test_data['label']=='spam'))

# Save the model
model.save('sms_spam_model.h5')

In [ ]:
# Model to predict probabily of messsage being "spam" or "ham"

def predict_message(pred_text):
    # Load the saved model
    model = keras.models.load_model('sms_spam_model.h5')
    
    # Preprocess the input text
    pred_sequence = tokenizer.texts_to_sequences([pred_text])
    pred_padded = pad_sequences(pred_sequence, maxlen=100, truncating='post', padding='post')
    
    # Make predictions and return the result
    pred_prob = (model.predict(pred_padded)[0][0])
    pred_label = 'ham' if pred_prob < 0.1 else 'spam'
    
    return [pred_prob, pred_label]

In [ ]:
# Message prediction tester QA
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("All messages were correctly identified")
  else:
    print("Mistakes were made")

test_predictions()